In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [9]:
from sklearn.datasets import fetch_olivetti_faces

In [10]:
from sklearn.preprocessing import LabelBinarizer

In [11]:
data = fetch_olivetti_faces(shuffle=True)
labels = LabelBinarizer().fit_transform(data.target)

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to C:\Users\ARPIT\scikit_learn_data


In [17]:
#training_Data
images_DataTraining = data.data[0:299]
labels_DataTraining = labels[0:299]

#Validation_Data
images_DataValidation = data.data[299:349]
labels_DataValidation = labels[299:349]

#testing_Data
images_DataTesting = data.data[349:399]
labels_DataTesting = labels[349:399]

## Convolution and Pooling layer

In [18]:
x = tf.placeholder(tf.float32 , [None , 4096])
y_ = tf.placeholder(tf.int32 , [None , 40])

#Add Function to create weight variable
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


#Add function to create bias variable
def bias_variable(shape):
    initial = tf.constant(0.1,shape = shape)
    return tf.Variable(initial)

#Add convolution function
def conv2d(x, W):
    return (tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding = "SAME"))

#Add pooling function
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding= "SAME")

#Reshape the data for inputting it to the input layer
x_image= tf.reshape(x, [-1, 64, 64, 1])

#create 1st convolutional and pooling layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image,  W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

print("Layer 1(CONV + POOLING)", h_pool1)

#create 2nd convolutional and pooling layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])


h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

print("Layer 2(CONV + POOLING)", h_pool2)

#create 3rd convolutional and pooling layer
W_conv3 = weight_variable([5, 5, 64, 128])
b_conv3 = bias_variable([128])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)

print("Layer 3(CONV + POOLING)", h_pool3)

#1st fully connected layer
W_fc1 = weight_variable([8*8*128, 1024])
b_fc1 = bias_variable([1024])

#Reshape the last convolutional layer output to connect with fully connected layer
h_pool2_flat = tf.reshape(h_pool3, [-1, 8*8*128])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+ b_fc1)

#Add dropout layer
rate = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, rate)

print('FC Layer 4', h_fc1_drop)
#output layer
W_fc2 = weight_variable([1024, 40])
b_fc2 = bias_variable([40])

y_conv = tf.matmul(h_fc1_drop, W_fc2)+ b_fc2

print('Output Layer 5', y_conv)

Layer 1(CONV + POOLING) Tensor("MaxPool_3:0", shape=(?, 32, 32, 32), dtype=float32)
Layer 2(CONV + POOLING) Tensor("MaxPool_4:0", shape=(?, 16, 16, 64), dtype=float32)
Layer 3(CONV + POOLING) Tensor("MaxPool_5:0", shape=(?, 8, 8, 128), dtype=float32)
FC Layer 4 Tensor("dropout_1/mul_1:0", shape=(?, 1024), dtype=float32)
Output Layer 5 Tensor("add_9:0", shape=(?, 40), dtype=float32)


## Training

In [19]:
#Network Base of Computation

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_ , logits=y_conv))
training_op = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#Prediction Measure
correct_prediction = tf.equal(tf.argmax(y_conv , 1),tf.arg_max(y_ , 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction , tf.float32))

saver = tf.train.Saver()

#Mention about the number of epochs training has to happen
n_epochs= 200

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epochs in range(n_epochs):
        sess.run(training_op, feed_dict = {x: images_DataTraining, y_: labels_DataTraining , rate:0.5})
        acc_train = accuracy.eval(feed_dict = {x: images_DataTraining, y_:labels_DataTraining, rate:1})
        acc_validation = accuracy.eval(feed_dict = {x: images_DataValidation, y_:labels_DataValidation, rate:0.5})
        print(epochs, "Train accuracy", acc_train, "validation accuracy", acc_validation)
    save_path = saver.save(sess, "./my_model_find.ckpt")

0 Train accuracy 0.02341137 validation accuracy 0.0
1 Train accuracy 0.013377926 validation accuracy 0.02
2 Train accuracy 0.026755853 validation accuracy 0.02
3 Train accuracy 0.050167225 validation accuracy 0.04
4 Train accuracy 0.05685619 validation accuracy 0.02
5 Train accuracy 0.050167225 validation accuracy 0.02
6 Train accuracy 0.040133778 validation accuracy 0.04
7 Train accuracy 0.050167225 validation accuracy 0.0
8 Train accuracy 0.050167225 validation accuracy 0.0
9 Train accuracy 0.06354515 validation accuracy 0.0
10 Train accuracy 0.06354515 validation accuracy 0.04
11 Train accuracy 0.07692308 validation accuracy 0.06
12 Train accuracy 0.080267556 validation accuracy 0.08
13 Train accuracy 0.09364548 validation accuracy 0.0
14 Train accuracy 0.10702341 validation accuracy 0.0
15 Train accuracy 0.117056854 validation accuracy 0.02
16 Train accuracy 0.12040134 validation accuracy 0.06
17 Train accuracy 0.12040134 validation accuracy 0.0
18 Train accuracy 0.12374582 validat

156 Train accuracy 1.0 validation accuracy 0.62
157 Train accuracy 1.0 validation accuracy 0.66
158 Train accuracy 1.0 validation accuracy 0.72
159 Train accuracy 1.0 validation accuracy 0.7
160 Train accuracy 1.0 validation accuracy 0.7
161 Train accuracy 1.0 validation accuracy 0.74
162 Train accuracy 1.0 validation accuracy 0.72
163 Train accuracy 1.0 validation accuracy 0.68
164 Train accuracy 1.0 validation accuracy 0.74
165 Train accuracy 1.0 validation accuracy 0.7
166 Train accuracy 1.0 validation accuracy 0.66
167 Train accuracy 1.0 validation accuracy 0.68
168 Train accuracy 1.0 validation accuracy 0.8
169 Train accuracy 1.0 validation accuracy 0.72
170 Train accuracy 1.0 validation accuracy 0.72
171 Train accuracy 1.0 validation accuracy 0.74
172 Train accuracy 1.0 validation accuracy 0.74
173 Train accuracy 1.0 validation accuracy 0.74
174 Train accuracy 1.0 validation accuracy 0.76
175 Train accuracy 1.0 validation accuracy 0.76
176 Train accuracy 1.0 validation accuracy 0

In [22]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_find.ckpt")
    acc_testing = accuracy.eval(feed_dict = {x: images_DataTesting, y_:labels_DataTesting, rate:1})
    print("Testing Accuracy", acc_testing)

Testing Accuracy 0.92
